In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash
!pip install transformers
!pip install spacy
!pip install yake

--2021-12-02 01:48:46--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2021-12-02 01:48:46--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-12-02 01:48:46--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:44

In [ ]:
import pickle
import sparknlp
import pandas as pd
from glob import glob
from pyspark.sql import SparkSession
from pyspark.sql.functions import month, year, concat, min, max, lower, udf
from pyspark.sql.types import IntegerType

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.common import *
from pyspark.ml.feature import VectorAssembler

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

spark = sparknlp.start()
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[4]")\
    .config("spark.driver.memory","16G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.3.2")\
    .getOrCreate()

In [ ]:
trending_news = spark.read.json('/content/drive/MyDrive/news_keywords_count.json')
trending_view_count = 10000

fp = [f for f in glob('/content/drive/**', recursive=True) if 'youtube_clean_' in f and '.json' in f][0]
df = spark.read.json(fp)
df = df.filter( (df.views>0) )
df = df.withColumn('label', (df.views>trending_view_count) )
df = df.withColumn('label', df.label.cast("integer") )
df = df.withColumn('year', year(df.publish_date_new))
df = df.withColumn('month', month(df.publish_date_new))
df = df.withColumn('year_month', concat(df.year,df.month))

1) Add combination painting tranding news on df
2) Add other features data

In [ ]:
trending_news.show(3)

+------------------+-----+----------+
|               col|count|year_month|
+------------------+-----+----------+
|benoit coeure said|    5|     20196|
| republican senate|   11|     20162|
|   united airlines|  208|     20174|
+------------------+-----+----------+
only showing top 3 rows



In [ ]:
df.show(3)

+------+--------------+--------------------+--------------------+--------------------+-------------+------------+--------------+---------+----------+--------------------+-----------------+------+--------------------+----------+-----------------+------------------+----------+----------------+---------+--------------------+--------------------+--------------------+--------+---------+----------+--------------------+-----+----+-----+----------+
|   _c0|age_restricted|              author|             channel|         description|duration_long|duration_med|duration_short|has_music|has_sounds|            keywords|language_detected|length|            metadata|n_keywords|n_posts_by_author|n_posts_by_channel|post_covid|publish_date_new|   rating|         stream_info|               title|          transcript|   views|views_cat|views_high|           watch_url|label|year|month|year_month|
+------+--------------+--------------------+--------------------+--------------------+-------------+----------

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col

window = Window.partitionBy(trending_news['year_month']).orderBy(trending_news['count'].desc())

trending_news_ranked = trending_news.select('*', rank().over(window).alias('rank')).filter(col('rank') <= 5) 

In [ ]:
trending_news_ranked.show(10)

+--------------------+-----+----------+----+
|                 col|count|year_month|rank|
+--------------------+-----+----------+----+
|            new york| 1398|     20183|   1|
|       united states| 1375|     20183|   2|
|         white house| 1160|     20183|   3|
|        donald trump| 1114|     20183|   4|
|beijing headline ...|  694|     20183|   5|
|            new york| 1539|     20175|   1|
|         white house| 1313|     20175|   2|
|        donald trump| 1131|     20175|   3|
|       united states| 1075|     20175|   4|
|president donald ...|  652|     20175|   5|
+--------------------+-----+----------+----+
only showing top 10 rows



In [ ]:
tnr1 = trending_news_ranked.filter(trending_news_ranked.rank==1).select(['year_month','col']).withColumnRenamed('col','trending_1')
tnr2 = trending_news_ranked.filter(trending_news_ranked.rank==2).select(['year_month','col']).withColumnRenamed('col','trending_2')
tnr3 = trending_news_ranked.filter(trending_news_ranked.rank==3).select(['year_month','col']).withColumnRenamed('col','trending_3')
tnr4 = trending_news_ranked.filter(trending_news_ranked.rank==4).select(['year_month','col']).withColumnRenamed('col','trending_4')
tnr5 = trending_news_ranked.filter(trending_news_ranked.rank==5).select(['year_month','col']).withColumnRenamed('col','trending_5')

In [ ]:
tnr_combined = tnr1.join(
    tnr2, on='year_month', how='outer').join(
        tnr3, on='year_month', how='outer').join(
            tnr4, on='year_month', how='outer').join(
                tnr5, on='year_month', how='outer')
            
tnr_combined = tnr_combined.withColumn("year_month", tnr_combined["year_month"].cast(IntegerType()))

In [ ]:
tnr_combined.select(min('year_month')).collect()[0]

Row(min(year_month)=20161)

In [ ]:
tnr_combined.select(max('year_month')).collect()[0]

Row(max(year_month)=201812)

In [ ]:
tnr_combined.filter(tnr_combined.year_month==201812).show()

+----------+----------+-------------+------------+-----------+--------------------+
|year_month|trending_1|   trending_2|  trending_3| trending_4|          trending_5|
+----------+----------+-------------+------------+-----------+--------------------+
|    201812|  new york|united states|donald trump|white house|      european union|
|    201812|  new york|united states|donald trump|white house|president donald ...|
+----------+----------+-------------+------------+-----------+--------------------+



In [ ]:
df_joined = df.join(tnr_combined, on='year_month', how='inner')

In [ ]:
df_joined.count()

89079

In [ ]:
df_joined.show(3)

+----------+------+--------------+--------------------+--------------------+--------------------+-------------+------------+--------------+---------+----------+--------+-----------------+------+--------------------+----------+-----------------+------------------+----------+----------------+------+--------------------+--------------------+--------------------+------+---------+----------+--------------------+-----+----+-----+-------------+----------+------------+-----------+--------------------+
|year_month|   _c0|age_restricted|              author|             channel|         description|duration_long|duration_med|duration_short|has_music|has_sounds|keywords|language_detected|length|            metadata|n_keywords|n_posts_by_author|n_posts_by_channel|post_covid|publish_date_new|rating|         stream_info|               title|          transcript| views|views_cat|views_high|           watch_url|label|year|month|   trending_1|trending_2|  trending_3| trending_4|          trending_5|
+-

In [ ]:
df_joined = df_joined.withColumn(
    'on_trend_title', 
    lower( df_joined.title ).contains( 
        df_joined.trending_1 )|lower( df_joined.title ).contains( 
            df_joined.trending_2 )|lower( df_joined.title ).contains( 
                df_joined.trending_3 )|lower( df_joined.title ).contains( 
                    df_joined.trending_4 )|lower( df_joined.title ).contains( 
                        df_joined.trending_5 )
)

df_joined = df_joined.withColumn(
    'on_trend_transcript', 
    lower( df_joined.transcript ).contains( 
        df_joined.trending_1 )|lower( df_joined.transcript ).contains( 
            df_joined.trending_2 )|lower( df_joined.transcript ).contains( 
                df_joined.trending_3 )|lower( df_joined.transcript ).contains( 
                    df_joined.trending_4 )|lower( df_joined.transcript ).contains( 
                        df_joined.trending_5 )
)
                    
df_joined = df_joined.withColumn(
    'on_trend_description', 
    lower( df_joined.description ).contains( 
        df_joined.trending_1 )|lower( df_joined.description ).contains( 
            df_joined.trending_2 )|lower( df_joined.description ).contains( 
                df_joined.trending_3 )|lower( df_joined.description ).contains( 
                    df_joined.trending_4 )|lower( df_joined.description ).contains( 
                        df_joined.trending_5 )
)

In [ ]:
train, test = df_joined.randomSplit([0.8, 0.2])

In [ ]:
document_assembler1 = DocumentAssembler()\
  .setInputCol("transcript")\
  .setOutputCol("document")
 
word_embeddings1 = BertSentenceEmbeddings.pretrained('sent_small_bert_L8_512', 'en')\
  .setInputCols(["document"])\
  .setOutputCol("transcript_embeddings")

document_assembler2 = DocumentAssembler()\
  .setInputCol("title")\
  .setOutputCol("document")
 
word_embeddings2 = BertSentenceEmbeddings.pretrained('sent_small_bert_L8_512', 'en')\
  .setInputCols(["document"])\
  .setOutputCol("title_embeddings")

document_assembler3 = DocumentAssembler()\
  .setInputCol("description")\
  .setOutputCol("document")
 
word_embeddings3 = BertSentenceEmbeddings.pretrained('sent_small_bert_L8_512', 'en')\
  .setInputCols(["document"])\
  .setOutputCol("description_embeddings")


clf1 = ClassifierDLApproach()\
  .setInputCols('transcript_embeddings')\
  .setOutputCol("prediction")\
  .setLabelColumn("label")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)

clf2 = ClassifierDLApproach()\
  .setInputCols('title_embeddings')\
  .setOutputCol("prediction")\
  .setLabelColumn("label")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)

clf3 = ClassifierDLApproach()\
  .setInputCols('description_embeddings')\
  .setOutputCol("prediction")\
  .setLabelColumn("label")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)

pipeline1 = Pipeline().setStages(
  [
    document_assembler1,
    word_embeddings1,
    clf1,
  ]
)
pipeline2 = Pipeline().setStages(
  [
    document_assembler2,
    word_embeddings2,
    clf2,
  ]
)
pipeline3 = Pipeline().setStages(
  [
    document_assembler3,
    word_embeddings3,
    clf3,
  ]
)

sent_small_bert_L8_512 download started this may take some time.
Approximate size to download 149.1 MB
[OK!]
sent_small_bert_L8_512 download started this may take some time.
Approximate size to download 149.1 MB
[OK!]
sent_small_bert_L8_512 download started this may take some time.
Approximate size to download 149.1 MB
[OK!]


In [40]:
cols = ['transcript','label' ]
model1 = pipeline1.fit(train.select( cols ) )
train_pred1 = model1.transform(train.select(cols) )
test_pred1 = model1.transform(test.select(cols))

cols = ['title','label' ]
model2 = pipeline2.fit(train.select( cols ) )
train_pred2 = model2.transform(train.select(cols) )
test_pred2 = model2.transform(test.select(cols))

cols = ['description','label' ]
model3 = pipeline3.fit(train.select( cols ) )
train_pred3 = model3.transform(train.select(cols) )
test_pred3 = model3.transform(test.select(cols))

In [41]:
train.show(2)

+----------+---+--------------+----------------+--------------------+--------------------+-------------+------------+--------------+---------+----------+--------------------+-----------------+------+--------+----------+-----------------+------------------+----------+----------------+---------+--------------------+--------------------+--------------------+--------+---------+----------+--------------------+-----+----+-----+-------------+----------+------------+-----------+--------------------+--------------+-------------------+--------------------+
|year_month|_c0|age_restricted|          author|             channel|         description|duration_long|duration_med|duration_short|has_music|has_sounds|            keywords|language_detected|length|metadata|n_keywords|n_posts_by_author|n_posts_by_channel|post_covid|publish_date_new|   rating|         stream_info|               title|          transcript|   views|views_cat|views_high|           watch_url|label|year|month|   trending_1|trending

In [42]:
def compile_final_model_inputs(pred1, pred2, pred3, feature_df):
    pred1_collection = [i[0][0] for i in pred1.select('prediction.result').collect()]
    pred2_collection = [i[0][0] for i in pred2.select('prediction.result').collect()]
    pred3_collection = [i[0][0] for i in pred3.select('prediction.result').collect()]
    other_features = feature_df.select(
        [
            'age_restricted','duration_long','duration_med','duration_short','has_music','has_sounds',
            'n_keywords', 'n_posts_by_author','n_posts_by_channel','post_covid', 'rating', 
            'on_trend_title', 'on_trend_description','on_trend_transcript'
        ]
    ).toPandas()
    input_df = pd.DataFrame()
    input_df['transcript_prediction'] = pred1_collection
    input_df['title_prediction'] = pred2_collection
    input_df['description_prediction'] = pred3_collection
    input_df = pd.concat([input_df, other_features], axis=1)
    return input_df

In [ ]:
input_df_train = compile_final_model_inputs(train_pred1, train_pred2, train_pred3, train)
input_df_test = compile_final_model_inputs(test_pred1, test_pred2, test_pred3, test)

In [ ]:
train_y = [i[0] for i in train.select('label').collect()]
test_y = [i[0] for i in test.select('label').collect()]

In [ ]:
pickle.dump([input_df_train, input_df_test, train_y, test_y], open('/content/drive/MyDrive/bdp_project_final_train_test.p', mode='wb') )
print('finished')